## 4.1 FewShotPromptTemplate

In [1]:
# api key 인증
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

True

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

# t = PromptTemplate.from_template('What is the capital of {country}')

# t.format(country='France')

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# example_template = """
#     Human: {question}
#     AI:{answer}
# """

# example_template = PromptTemplate.from_template(example_template)
example_prompt = PromptTemplate.from_template('Human : {question}\nAI:{answer}')

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human : What do you know about {country}?",
    input_variables=['country']
)

prompt.format(country='Germany')

chain = prompt | chat

chain.invoke({
    'country':'Korea'
})

AI:
        I know this:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean Won

AIMessageChunk(content='AI:\n        I know this:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean Won')

- 예제들을 가져오고 예제들을 prompt를 사용해서 형식화 해주고 그런 다음 이를 fewShotPromptTemplate에 전달
- fewShotPromptTemplate은 각각 예제들을 가져와 example prompt를 이용하여 형식화 해주고 그 다음 마지막으로 내용 마지막에 질문을 넣어줌

## 4.2 FewShotChatMessagePromptTemplate

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

# t = PromptTemplate.from_template('What is the capital of {country}')

# t.format(country='France')

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# example_template = """
#     Human: {question}
#     AI:{answer}
# """

# example_template = PromptTemplate.from_template(example_template)
example_prompt = ChatPromptTemplate.from_messages([
    ('human', 'What do you know about {country}?'),
    ('ai', '{answer}')
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a geography expert'),
    example_prompt,
    ('human', 'What do you know about {country}?')
])

chain = final_prompt | chat

chain.invoke({
    'country':'Korea'
})


        I know this:
        There are two Koreas - North Korea and South Korea
        Capital of South Korea: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean Won
        

AIMessageChunk(content='\n        I know this:\n        There are two Koreas - North Korea and South Korea\n        Capital of South Korea: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean Won\n        ')

## 4.3 LengthBasedExampleSelector

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

# t = PromptTemplate.from_template('What is the capital of {country}')

# t.format(country='France')

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# example_template = """
#     Human: {question}
#     AI:{answer}
# """

# example_template = PromptTemplate.from_template(example_template)
example_prompt = PromptTemplate.from_template('Human : {question}\nAI:{answer}')

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=10,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human : What do you know about {country}?",
    input_variables=['country']
)

prompt.format(country='Germany')


'Human : What do you know about Germany?'

- 예제 sample 만들기

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

# t = PromptTemplate.from_template('What is the capital of {country}')

# t.format(country='France')

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):
    
    def __init__(self,examples):
        self.examples = examples
        
    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import sample
        
        return sample(self.examples,k=1)

# example_template = """
#     Human: {question}
#     AI:{answer}
# """

# example_template = PromptTemplate.from_template(example_template)
example_prompt = PromptTemplate.from_template('Human : {question}\nAI:{answer}')

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human : What do you know about {country}?",
    input_variables=['country']
)

prompt.format(country='Germany')


'Human : What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman : What do you know about Germany?'

## 4.4 Serialization and Composition

- prompt load 하는 방법

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

# prompt = load_prompt('./prompt.json') # json 파일 형식
prompt = load_prompt('./prompt.yaml') # yaml 파일 형식

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

prompt.format(country = 'Korea')

'what is the capital of Korea'

- 많은 prompt들의 memory 등을 다 모으는 방법

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

full_prompt.format(
    character = 'Pirate',
    example_question = 'What is your location?',
    example_answer = 'Arrrrg! That is a secret! Arg arg!!',
    question = 'What is your favorite foods?',
)


'\n    \n    You are a role playing assistant.\n    And you are impersonating a Pirate\n\n                                     \n    \n    This is an example of how you talk:\n\n    Human: What is your location?\n    You: Arrrrg! That is a secret! Arg arg!!\n\n                              \n    \n    Start now!\n\n    Human: What is your favorite foods?\n    You:\n\n'

- chain 불러오기

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothing beats the taste o' the sea! Arrr!

AIMessageChunk(content="Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothing beats the taste o' the sea! Arrr!")

## 4.5 Caching

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(InMemoryCache()) # 모든 response가 메모리에 저장
set_debug(True) # log 같은게 보임

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ]
)

chat.predict('How to make Italin pasta?') # 3.6초 걸림

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How to make Italin pasta?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [2.72s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere's how to make it:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for about 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is about 1/8 inch thick.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted wa

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere's how to make it:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for about 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is about 1/8 inch thick.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for about 2-3 minutes, or until al dente.\n9. Drain the pasta and toss it with your favorite sauce, such as marinara, alfredo, or pesto.\n10. Serve hot and enjoy your homemade Italian pasta!"

- cache을 이용하면 돈을 절약할 수 있음 왜냐하면 계속 반복되는 질문들에 대해서 답변을 새로 생성하는게 아니라 저장되게 하는 방법이기 때문

In [13]:
chat.predict('How to make Italin pasta?') # 0초 걸림

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere's how to make it:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for about 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is about 1/8 inch thick.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for about 2-3 minutes, or until al dente.\n9. Drain the pasta and toss it with your favorite sauce, such as marinara, Alfredo, or pesto.\n10. Serve hot and enjoy your homemade Italian pasta!"

- 컴퓨터 데이터베이스에 저장

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache('cache db')) # 컴퓨터 내 데이터에 저장

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ]
)

chat.predict('How to make Italin pasta?') # 3.6초 걸림

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere's how to make it:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for about 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is about 1/8 inch thick.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for about 2-3 minutes, or until al dente.\n9. Drain the pasta and toss it with your favorite sauce, such as marinara, pesto, or Alfredo.\n\nEnjoy your homemade Italian pasta!"

## 4.6 Serialization

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1
)
with get_openai_callback() as usage:
    chat.predict('What is recipe for beer?')
    print(usage)

Tokens Used: 183
	Prompt Tokens: 13
	Completion Tokens: 170
Successful Requests: 1
Total Cost (USD): $0.0003595


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1
)

with get_openai_callback() as usage:
    a = chat.predict('What is recipe for beer?')
    b = chat.predict('What is recipe for wine?')
    print(a,b,'\n')
    print(usage)

Ingredients:
- Water
- Barley malt
- Hops
- Yeast

Instructions:
1. Malting: Barley grains are soaked in water, germinated, and then dried in a kiln to produce malted barley.
2. Mashing: The malted barley is crushed and mixed with hot water to extract sugars. This mixture is then strained to create a liquid called wort.
3. Boiling: The wort is boiled and hops are added to provide bitterness and aroma to the beer.
4. Fermentation: The boiled wort is cooled and yeast is added to ferment the sugars into alcohol and carbon dioxide.
5. Conditioning: The beer is aged for a period of time to allow flavors to develop and mature.
6. Packaging: The beer is filtered, carbonated, and packaged for distribution and consumption. Ingredients:
- Grapes (red or white)
- Yeast
- Sugar

Instructions:
1. Harvest the grapes and remove them from the stems.
2. Crush the grapes to release the juice.
3. Add yeast to the grape juice to start the fermentation process.
4. Stir in sugar to feed the yeast and increa

- Serialization(llm)

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI

chat = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model='gpt-3.5-turbo-16'
)

chat.save('model.json')

- 모델 불러오기(llm)

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

chat = load_llm('model.json')

chat

c:\Users\yjroh\Documents\Python\Self-Study\AI\env\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\yjroh\Documents\Python\Self-Study\AI\env\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-16', model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})